In [52]:
!pip install pathway --quiet


In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [54]:
df = pd.read_csv('dataset.csv')
df

# You can find the sample dataset here: https://drive.google.com/file/d/1D479FLjp9aO3Mg8g6Lpj9oRViWacurA6/view?usp=sharing

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [55]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)
df.QueueLength.unique()

array([ 1,  2,  3,  4,  5,  7,  6,  9,  8, 10, 11, 12, 13, 14,  0, 15])

In [56]:
# Save the selected columns to a CSV file for streaming or downstream processing
df[["Timestamp", "Occupancy", "Capacity"]].to_csv("parking_stream.csv", index=False)
df[["Timestamp", "Occupancy", "Capacity","IsSpecialDay","VehicleType","QueueLength","TrafficConditionNearby"]].to_csv("parking_stream2.csv", index=False)
# Note: Only three features are used here for simplicity.
# Participants are expected to incorporate additional relevant features in their models.

In [57]:
f = pd.read_csv('parking_stream2.csv')

traffic_map = {"low": 1, "average":2 ,"high": 3}
f["TrafficLevel"] = f["TrafficConditionNearby"].map(traffic_map)
f.drop(columns=["TrafficConditionNearby"],inplace = True)
f.to_csv("parking_stream_2.csv", index=False)
f

,Timestamp,Occupancy,Capacity,IsSpecialDay,VehicleType,QueueLength,TrafficLevel
0,2016-10-04 07:59:00,61,577,0,car,1,1
1,2016-10-04 07:59:00,237,1200,0,bike,2,1
2,2016-10-04 07:59:00,264,687,0,car,2,1
3,2016-10-04 07:59:00,249,485,0,car,2,1
4,2016-10-04 07:59:00,614,1920,0,cycle,2,1
...,...,...,...,...,...,...,...
18363,2016-12-19 16:30:00,373,470,0,car,2,1
18364,2016-12-19 16:30:00,387,387,0,car,2,1
18365,2016-12-19 16:30:00,193,577,0,cycle,2,1
18366,2016-12-19 16:30:00,1671,3103,0,car,3,1


In [58]:
d1 = pd.read_csv('dataset.csv')
d1['Timestamp'] = pd.to_datetime(d1['LastUpdatedDate'] + ' ' + d1['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
d1= d1.sort_values('Timestamp').reset_index(drop=True)
d1 = d1.drop(columns = ["LastUpdatedDate","LastUpdatedTime"])
d1['day_of_month'] = d1['Timestamp'].dt.day
d1['day_of_week'] = d1['Timestamp'].dt.dayofweek
d1["hour"] = d1.Timestamp.dt.hour
d1["minute"] = d1.Timestamp.dt.minute
d1["min_30"] = (d1["minute"] == 30).astype(int)
d1["min_59"] = (d1["minute"] == 59).astype(int)
traffic_map = {"low": 1, "average":2 ,"high": 3}
d1["TrafficConditionNearby"] = d1["TrafficConditionNearby"].map(traffic_map)
weights = {
    'cycle': 10,    # avg. bicycle weight ~15 kg
    'bike': 30,    # avg. motorcycle weight ~150 kg
    'car': 80,    # avg. car curb weight ~1500 kg
    'truck': 150   # avg. light‑duty truck ~8000 kg
}
d1['VehicleWeight'] = d1['VehicleType'].map(weights)

max_wt = d1['VehicleWeight'].max()
d1['VehicleWeightNorm'] = d1['VehicleWeight'] / max_wt

d1 = d1.drop(['VehicleType', 'Latitude', 'Longitude','VehicleWeight'], axis=1)
d1['Fraction_Occupied'] = d1['Occupancy'] / d1['Capacity']
d1 = d1.drop(['ID'], axis=1)
d1[["Timestamp","Occupancy","Capacity","QueueLength","SystemCodeNumber","IsSpecialDay","VehicleWeightNorm","TrafficConditionNearby"]].to_csv("parking_stream2.csv", index=False)

In [59]:
ff= pd.read_csv('parking_stream2.csv')
ff['SystemCodeNumber'].unique()

array(['BHMBCCMKT01', 'BHMNCPHST01', 'BHMMBMMBX01', 'BHMNCPNST01',
       'Shopping', 'BHMEURBRD01', 'Broad Street', 'Others-CCCPS8',
       'Others-CCCPS105a', 'Others-CCCPS119a', 'BHMBCCTHL01',
       'Others-CCCPS135a', 'Others-CCCPS202', 'Others-CCCPS98'],
      dtype=object)

In [60]:
ff[ff['SystemCodeNumber']=="BHMBCCMKT01"].to_csv("parking_stream_a.csv",index=False)
ff[ff['SystemCodeNumber']=="BHMNCPHST01"].to_csv("parking_stream_b.csv",index=False)
ff[ff['SystemCodeNumber']=="BHMMBMMBX01"].to_csv("parking_stream_c.csv",index=False)
ff[ff['SystemCodeNumber']=="BHMNCPNST01"].to_csv("parking_stream_d.csv",index=False)
ff[ff['SystemCodeNumber']=="Shopping"].to_csv("parking_stream_e.csv",index=False)
ff[ff['SystemCodeNumber']=="BHMEURBRD01"].to_csv("parking_stream_f.csv",index=False)
ff[ff['SystemCodeNumber']=="Broad Street"].to_csv("parking_stream_g.csv",index=False)
ff[ff['SystemCodeNumber']=="Others-CCCPS8"].to_csv("parking_stream_h.csv",index=False)
ff[ff['SystemCodeNumber']=="Others-CCCPS105a"].to_csv("parking_stream_i.csv",index=False)
ff[ff['SystemCodeNumber']=="Others-CCCPS119a"].to_csv("parking_stream_j.csv",index=False)
ff[ff['SystemCodeNumber']=="BHMBCCTHL01"].to_csv("parking_stream_k.csv",index=False)
ff[ff['SystemCodeNumber']=="Others-CCCPS135a"].to_csv("parking_stream_l.csv",index=False)
ff[ff['SystemCodeNumber']=="Others-CCCPS202"].to_csv("parking_stream_m.csv",index=False)
ff[ff['SystemCodeNumber']=="Others-CCCPS98"].to_csv("parking_stream_n.csv",index=False)


In [61]:
# === 1) Imports ===
import pathway as pw
import datetime

# === 2) Schema matching your CSV ===
class ParkingBataSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    IsSpecialDay: int
    QueueLength: int
    VehicleWeightNorm: float
    TrafficConditionNearby: int

# === 3) Load data as streaming replay ===
bata = pw.demo.replay_csv(
    "parking_stream_a.csv",
    schema=ParkingBataSchema,
    input_rate=100
)

# === 4) Parse timestamp and derive day column ===
fmt = "%Y-%m-%d %H:%M:%S"
bata_with_time = bata.with_columns(
    t = bata.Timestamp.dt.strptime(fmt),
    day = bata.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# === 5) Define daily tumbling window and compute only price ===
delta_window = (
    bata_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        queue_length=pw.reducers.max(pw.this.QueueLength),
        traffic_condition_nearby=pw.reducers.max(pw.this.TrafficConditionNearby),
        vehicle_weight_norm=pw.reducers.max(pw.this.VehicleWeightNorm),
    )
    .with_columns(
        price = 10 * (1 + 0.5 * (
            0.8 * ((pw.this.occ_max - pw.this.occ_min) / pw.this.cap)
            + 0.4 * pw.this.is_special_day
            + 0.05 * pw.this.queue_length
            - 0.3 * pw.this.traffic_condition_nearby
            + 0.2 * pw.this.vehicle_weight_norm
        ))
    )
)


In [62]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime"
        ,  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [63]:
%%capture --no-display
pw.run()

Output()

In [64]:
# === 1) Imports ===
import pathway as pw
import datetime

# === 2) Schema matching your CSV ===
class ParkingBataSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    IsSpecialDay: int
    QueueLength: int
    VehicleWeightNorm: float
    TrafficConditionNearby: int

# === 3) Load data as streaming replay ===
bata = pw.demo.replay_csv(
    "parking_stream_b.csv",
    schema=ParkingBataSchema,
    input_rate=100
)

# === 4) Parse timestamp and derive day column ===
fmt = "%Y-%m-%d %H:%M:%S"
bata_with_time = bata.with_columns(
    t = bata.Timestamp.dt.strptime(fmt),
    day = bata.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# === 5) Define daily tumbling window and compute only price ===
delta_window = (
    bata_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        queue_length=pw.reducers.max(pw.this.QueueLength),
        traffic_condition_nearby=pw.reducers.max(pw.this.TrafficConditionNearby),
        vehicle_weight_norm=pw.reducers.max(pw.this.VehicleWeightNorm),
    )
    .with_columns(
        price = 10 * (1 + 0.5 * (
            0.8 * ((pw.this.occ_max - pw.this.occ_min) / pw.this.cap)
            + 0.4 * pw.this.is_special_day
            + 0.05 * pw.this.queue_length
            - 0.3 * pw.this.traffic_condition_nearby
            + 0.2 * pw.this.vehicle_weight_norm
        ))
    )
)
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime"
        ,  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()


Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [ ]:
%%capture --no-display
pw.run()

Output()

In [ ]:
# === 1) Imports ===
import pathway as pw
import datetime

# === 2) Schema matching your CSV ===
class ParkingBataSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    IsSpecialDay: int
    QueueLength: int
    VehicleWeightNorm: float
    TrafficConditionNearby: int

# === 3) Load data as streaming replay ===
bata = pw.demo.replay_csv(
    "parking_stream_c.csv",
    schema=ParkingBataSchema,
    input_rate=100
)

# === 4) Parse timestamp and derive day column ===
fmt = "%Y-%m-%d %H:%M:%S"
bata_with_time = bata.with_columns(
    t = bata.Timestamp.dt.strptime(fmt),
    day = bata.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# === 5) Define daily tumbling window and compute only price ===
delta_window = (
    bata_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        queue_length=pw.reducers.max(pw.this.QueueLength),
        traffic_condition_nearby=pw.reducers.max(pw.this.TrafficConditionNearby),
        vehicle_weight_norm=pw.reducers.max(pw.this.VehicleWeightNorm),
    )
    .with_columns(
        price = 10 * (1 + 0.5 * (
            0.8 * ((pw.this.occ_max - pw.this.occ_min) / pw.this.cap)
            + 0.4 * pw.this.is_special_day
            + 0.05 * pw.this.queue_length
            - 0.3 * pw.this.traffic_condition_nearby
            + 0.2 * pw.this.vehicle_weight_norm
        ))
    )
)
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime"
        ,  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()


In [ ]:
%%capture --no-display
pw.run()

In [ ]:
# === 1) Imports ===
import pathway as pw
import datetime

# === 2) Schema matching your CSV ===
class ParkingBataSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    IsSpecialDay: int
    QueueLength: int
    VehicleWeightNorm: float
    TrafficConditionNearby: int

# === 3) Load data as streaming replay ===
bata = pw.demo.replay_csv(
    "parking_stream_d.csv",
    schema=ParkingBataSchema,
    input_rate=100
)

# === 4) Parse timestamp and derive day column ===
fmt = "%Y-%m-%d %H:%M:%S"
bata_with_time = bata.with_columns(
    t = bata.Timestamp.dt.strptime(fmt),
    day = bata.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# === 5) Define daily tumbling window and compute only price ===
delta_window = (
    bata_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        queue_length=pw.reducers.max(pw.this.QueueLength),
        traffic_condition_nearby=pw.reducers.max(pw.this.TrafficConditionNearby),
        vehicle_weight_norm=pw.reducers.max(pw.this.VehicleWeightNorm),
    )
    .with_columns(
        price = 10 * (1 + 0.5 * (
            0.8 * ((pw.this.occ_max - pw.this.occ_min) / pw.this.cap)
            + 0.4 * pw.this.is_special_day
            + 0.05 * pw.this.queue_length
            - 0.3 * pw.this.traffic_condition_nearby
            + 0.2 * pw.this.vehicle_weight_norm
        ))
    )
)
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime"
        ,  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()


In [ ]:
%%capture --no-display
pw.run()

In [ ]:
# === 1) Imports ===
import pathway as pw
import datetime

# === 2) Schema matching your CSV ===
class ParkingBataSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    IsSpecialDay: int
    QueueLength: int
    VehicleWeightNorm: float
    TrafficConditionNearby: int

# === 3) Load data as streaming replay ===
bata = pw.demo.replay_csv(
    "parking_stream_e.csv",
    schema=ParkingBataSchema,
    input_rate=100
)

# === 4) Parse timestamp and derive day column ===
fmt = "%Y-%m-%d %H:%M:%S"
bata_with_time = bata.with_columns(
    t = bata.Timestamp.dt.strptime(fmt),
    day = bata.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# === 5) Define daily tumbling window and compute only price ===
delta_window = (
    bata_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        queue_length=pw.reducers.max(pw.this.QueueLength),
        traffic_condition_nearby=pw.reducers.max(pw.this.TrafficConditionNearby),
        vehicle_weight_norm=pw.reducers.max(pw.this.VehicleWeightNorm),
    )
    .with_columns(
        price = 10 * (1 + 0.5 * (
            0.8 * ((pw.this.occ_max - pw.this.occ_min) / pw.this.cap)
            + 0.4 * pw.this.is_special_day
            + 0.05 * pw.this.queue_length
            - 0.3 * pw.this.traffic_condition_nearby
            + 0.2 * pw.this.vehicle_weight_norm
        ))
    )
)
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime"
        ,  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()


In [ ]:
%%capture --no-display
pw.run()

In [ ]:
# === 1) Imports ===
import pathway as pw
import datetime

# === 2) Schema matching your CSV ===
class ParkingBataSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    IsSpecialDay: int
    QueueLength: int
    VehicleWeightNorm: float
    TrafficConditionNearby: int

# === 3) Load data as streaming replay ===
bata = pw.demo.replay_csv(
    "parking_stream_f.csv",
    schema=ParkingBataSchema,
    input_rate=100
)

# === 4) Parse timestamp and derive day column ===
fmt = "%Y-%m-%d %H:%M:%S"
bata_with_time = bata.with_columns(
    t = bata.Timestamp.dt.strptime(fmt),
    day = bata.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# === 5) Define daily tumbling window and compute only price ===
delta_window = (
    bata_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        queue_length=pw.reducers.max(pw.this.QueueLength),
        traffic_condition_nearby=pw.reducers.max(pw.this.TrafficConditionNearby),
        vehicle_weight_norm=pw.reducers.max(pw.this.VehicleWeightNorm),
    )
    .with_columns(
        price = 10 * (1 + 0.5 * (
            0.8 * ((pw.this.occ_max - pw.this.occ_min) / pw.this.cap)
            + 0.4 * pw.this.is_special_day
            + 0.05 * pw.this.queue_length
            - 0.3 * pw.this.traffic_condition_nearby
            + 0.2 * pw.this.vehicle_weight_norm
        ))
    )
)
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime"
        ,  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()


In [ ]:
%%capture --no-display
pw.run()

In [ ]:
# === 1) Imports ===
import pathway as pw
import datetime

# === 2) Schema matching your CSV ===
class ParkingBataSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    IsSpecialDay: int
    QueueLength: int
    VehicleWeightNorm: float
    TrafficConditionNearby: int

# === 3) Load data as streaming replay ===
bata = pw.demo.replay_csv(
    "parking_stream_g.csv",
    schema=ParkingBataSchema,
    input_rate=100
)

# === 4) Parse timestamp and derive day column ===
fmt = "%Y-%m-%d %H:%M:%S"
bata_with_time = bata.with_columns(
    t = bata.Timestamp.dt.strptime(fmt),
    day = bata.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# === 5) Define daily tumbling window and compute only price ===
delta_window = (
    bata_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        queue_length=pw.reducers.max(pw.this.QueueLength),
        traffic_condition_nearby=pw.reducers.max(pw.this.TrafficConditionNearby),
        vehicle_weight_norm=pw.reducers.max(pw.this.VehicleWeightNorm),
    )
    .with_columns(
        price = 10 * (1 + 0.5 * (
            0.8 * ((pw.this.occ_max - pw.this.occ_min) / pw.this.cap)
            + 0.4 * pw.this.is_special_day
            + 0.05 * pw.this.queue_length
            - 0.3 * pw.this.traffic_condition_nearby
            + 0.2 * pw.this.vehicle_weight_norm
        ))
    )
)
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime"
        ,  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()


In [ ]:
%%capture --no-display
pw.run()

In [ ]:
# === 1) Imports ===
import pathway as pw
import datetime

# === 2) Schema matching your CSV ===
class ParkingBataSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    IsSpecialDay: int
    QueueLength: int
    VehicleWeightNorm: float
    TrafficConditionNearby: int

# === 3) Load data as streaming replay ===
bata = pw.demo.replay_csv(
    "parking_stream_h.csv",
    schema=ParkingBataSchema,
    input_rate=100
)

# === 4) Parse timestamp and derive day column ===
fmt = "%Y-%m-%d %H:%M:%S"
bata_with_time = bata.with_columns(
    t = bata.Timestamp.dt.strptime(fmt),
    day = bata.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# === 5) Define daily tumbling window and compute only price ===
delta_window = (
    bata_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        queue_length=pw.reducers.max(pw.this.QueueLength),
        traffic_condition_nearby=pw.reducers.max(pw.this.TrafficConditionNearby),
        vehicle_weight_norm=pw.reducers.max(pw.this.VehicleWeightNorm),
    )
    .with_columns(
        price = 10 * (1 + 0.5 * (
            0.8 * ((pw.this.occ_max - pw.this.occ_min) / pw.this.cap)
            + 0.4 * pw.this.is_special_day
            + 0.05 * pw.this.queue_length
            - 0.3 * pw.this.traffic_condition_nearby
            + 0.2 * pw.this.vehicle_weight_norm
        ))
    )
)
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime"
        ,  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()


In [ ]:
%%capture --no-display
pw.run()

In [ ]:
# === 1) Imports ===
import pathway as pw
import datetime

# === 2) Schema matching your CSV ===
class ParkingBataSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    IsSpecialDay: int
    QueueLength: int
    VehicleWeightNorm: float
    TrafficConditionNearby: int

# === 3) Load data as streaming replay ===
bata = pw.demo.replay_csv(
    "parking_stream_i.csv",
    schema=ParkingBataSchema,
    input_rate=100
)

# === 4) Parse timestamp and derive day column ===
fmt = "%Y-%m-%d %H:%M:%S"
bata_with_time = bata.with_columns(
    t = bata.Timestamp.dt.strptime(fmt),
    day = bata.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# === 5) Define daily tumbling window and compute only price ===
delta_window = (
    bata_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        queue_length=pw.reducers.max(pw.this.QueueLength),
        traffic_condition_nearby=pw.reducers.max(pw.this.TrafficConditionNearby),
        vehicle_weight_norm=pw.reducers.max(pw.this.VehicleWeightNorm),
    )
    .with_columns(
        price = 10 * (1 + 0.5 * (
            0.8 * ((pw.this.occ_max - pw.this.occ_min) / pw.this.cap)
            + 0.4 * pw.this.is_special_day
            + 0.05 * pw.this.queue_length
            - 0.3 * pw.this.traffic_condition_nearby
            + 0.2 * pw.this.vehicle_weight_norm
        ))
    )
)
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime"
        ,  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()


In [ ]:
%%capture --no-display
pw.run()

In [ ]:
# === 1) Imports ===
import pathway as pw
import datetime

# === 2) Schema matching your CSV ===
class ParkingBataSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    IsSpecialDay: int
    QueueLength: int
    VehicleWeightNorm: float
    TrafficConditionNearby: int

# === 3) Load data as streaming replay ===
bata = pw.demo.replay_csv(
    "parking_stream_j.csv",
    schema=ParkingBataSchema,
    input_rate=100
)

# === 4) Parse timestamp and derive day column ===
fmt = "%Y-%m-%d %H:%M:%S"
bata_with_time = bata.with_columns(
    t = bata.Timestamp.dt.strptime(fmt),
    day = bata.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# === 5) Define daily tumbling window and compute only price ===
delta_window = (
    bata_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        queue_length=pw.reducers.max(pw.this.QueueLength),
        traffic_condition_nearby=pw.reducers.max(pw.this.TrafficConditionNearby),
        vehicle_weight_norm=pw.reducers.max(pw.this.VehicleWeightNorm),
    )
    .with_columns(
        price = 10 * (1 + 0.5 * (
            0.8 * ((pw.this.occ_max - pw.this.occ_min) / pw.this.cap)
            + 0.4 * pw.this.is_special_day
            + 0.05 * pw.this.queue_length
            - 0.3 * pw.this.traffic_condition_nearby
            + 0.2 * pw.this.vehicle_weight_norm
        ))
    )
)
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime"
        ,  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()


In [ ]:
%%capture --no-display
pw.run()

In [ ]:
# === 1) Imports ===
import pathway as pw
import datetime

# === 2) Schema matching your CSV ===
class ParkingBataSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    IsSpecialDay: int
    QueueLength: int
    VehicleWeightNorm: float
    TrafficConditionNearby: int

# === 3) Load data as streaming replay ===
bata = pw.demo.replay_csv(
    "parking_stream_k.csv",
    schema=ParkingBataSchema,
    input_rate=100
)

# === 4) Parse timestamp and derive day column ===
fmt = "%Y-%m-%d %H:%M:%S"
bata_with_time = bata.with_columns(
    t = bata.Timestamp.dt.strptime(fmt),
    day = bata.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# === 5) Define daily tumbling window and compute only price ===
delta_window = (
    bata_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        queue_length=pw.reducers.max(pw.this.QueueLength),
        traffic_condition_nearby=pw.reducers.max(pw.this.TrafficConditionNearby),
        vehicle_weight_norm=pw.reducers.max(pw.this.VehicleWeightNorm),
    )
    .with_columns(
        price = 10 * (1 + 0.5 * (
            0.8 * ((pw.this.occ_max - pw.this.occ_min) / pw.this.cap)
            + 0.4 * pw.this.is_special_day
            + 0.05 * pw.this.queue_length
            - 0.3 * pw.this.traffic_condition_nearby
            + 0.2 * pw.this.vehicle_weight_norm
        ))
    )
)
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime"
        ,  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()


In [ ]:
%%capture --no-display
pw.run()

In [ ]:
# === 1) Imports ===
import pathway as pw
import datetime

# === 2) Schema matching your CSV ===
class ParkingBataSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    IsSpecialDay: int
    QueueLength: int
    VehicleWeightNorm: float
    TrafficConditionNearby: int

# === 3) Load data as streaming replay ===
bata = pw.demo.replay_csv(
    "parking_stream_l.csv",
    schema=ParkingBataSchema,
    input_rate=100
)

# === 4) Parse timestamp and derive day column ===
fmt = "%Y-%m-%d %H:%M:%S"
bata_with_time = bata.with_columns(
    t = bata.Timestamp.dt.strptime(fmt),
    day = bata.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# === 5) Define daily tumbling window and compute only price ===
delta_window = (
    bata_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        queue_length=pw.reducers.max(pw.this.QueueLength),
        traffic_condition_nearby=pw.reducers.max(pw.this.TrafficConditionNearby),
        vehicle_weight_norm=pw.reducers.max(pw.this.VehicleWeightNorm),
    )
    .with_columns(
        price = 10 * (1 + 0.5 * (
            0.8 * ((pw.this.occ_max - pw.this.occ_min) / pw.this.cap)
            + 0.4 * pw.this.is_special_day
            + 0.05 * pw.this.queue_length
            - 0.3 * pw.this.traffic_condition_nearby
            + 0.2 * pw.this.vehicle_weight_norm
        ))
    )
)
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime"
        ,  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()


In [ ]:
%%capture --no-display
pw.run()

In [ ]:
# === 1) Imports ===
import pathway as pw
import datetime

# === 2) Schema matching your CSV ===
class ParkingBataSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    IsSpecialDay: int
    QueueLength: int
    VehicleWeightNorm: float
    TrafficConditionNearby: int

# === 3) Load data as streaming replay ===
bata = pw.demo.replay_csv(
    "parking_stream_m.csv",
    schema=ParkingBataSchema,
    input_rate=100
)

# === 4) Parse timestamp and derive day column ===
fmt = "%Y-%m-%d %H:%M:%S"
bata_with_time = bata.with_columns(
    t = bata.Timestamp.dt.strptime(fmt),
    day = bata.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# === 5) Define daily tumbling window and compute only price ===
delta_window = (
    bata_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        queue_length=pw.reducers.max(pw.this.QueueLength),
        traffic_condition_nearby=pw.reducers.max(pw.this.TrafficConditionNearby),
        vehicle_weight_norm=pw.reducers.max(pw.this.VehicleWeightNorm),
    )
    .with_columns(
        price = 10 * (1 + 0.5 * (
            0.8 * ((pw.this.occ_max - pw.this.occ_min) / pw.this.cap)
            + 0.4 * pw.this.is_special_day
            + 0.05 * pw.this.queue_length
            - 0.3 * pw.this.traffic_condition_nearby
            + 0.2 * pw.this.vehicle_weight_norm
        ))
    )
)
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime"
        ,  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()


In [ ]:
%%capture --no-display
pw.run()

In [ ]:
# === 1) Imports ===
import pathway as pw
import datetime

# === 2) Schema matching your CSV ===
class ParkingBataSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    IsSpecialDay: int
    QueueLength: int
    VehicleWeightNorm: float
    TrafficConditionNearby: int

# === 3) Load data as streaming replay ===
bata = pw.demo.replay_csv(
    "parking_stream_n.csv",
    schema=ParkingBataSchema,
    input_rate=100
)

# === 4) Parse timestamp and derive day column ===
fmt = "%Y-%m-%d %H:%M:%S"
bata_with_time = bata.with_columns(
    t = bata.Timestamp.dt.strptime(fmt),
    day = bata.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# === 5) Define daily tumbling window and compute only price ===
delta_window = (
    bata_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ_max=pw.reducers.max(pw.this.Occupancy),
        occ_min=pw.reducers.min(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        is_special_day=pw.reducers.max(pw.this.IsSpecialDay),
        queue_length=pw.reducers.max(pw.this.QueueLength),
        traffic_condition_nearby=pw.reducers.max(pw.this.TrafficConditionNearby),
        vehicle_weight_norm=pw.reducers.max(pw.this.VehicleWeightNorm),
    )
    .with_columns(
        price = 10 * (1 + 0.5 * (
            0.8 * ((pw.this.occ_max - pw.this.occ_min) / pw.this.cap)
            + 0.4 * pw.this.is_special_day
            + 0.05 * pw.this.queue_length
            - 0.3 * pw.this.traffic_condition_nearby
            + 0.2 * pw.this.vehicle_weight_norm
        ))
    )
)
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title="Pathway: Daily Parking Price",
        x_axis_type="datetime"
        ,  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.circle("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()


In [ ]:
%%capture --no-display
pw.run()